In [1]:
import os
from twilio.rest import Client
from twilio_config import TWILIO_ACCOUNT_SID,TWILIO_AUTH_TOKEN,PHONE_NUMBER,API_KEY_WAPI
import time

from requests import Request, Session
from requests.exceptions import ConnectionError,Timeout, TooManyRedirects
import json

import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

from datetime import datetime

In [2]:
# query = 'Bogota'
query = 'Cali'
api_key = API_KEY_WAPI
url_clima = 'http://api.weatherapi.com/v1/forecast.json?key={}&q={}&days=1&aqi=no&alerts=no'.format(api_key,query)
response = requests.get(url_clima).json()

In [3]:
response

{'location': {'name': 'Cali',
  'region': 'Valle del Cauca',
  'country': 'Colombia',
  'lat': 3.44,
  'lon': -76.52,
  'tz_id': 'America/Bogota',
  'localtime_epoch': 1705349414,
  'localtime': '2024-01-15 15:10'},
 'current': {'last_updated_epoch': 1705348800,
  'last_updated': '2024-01-15 15:00',
  'temp_c': 31.0,
  'temp_f': 87.8,
  'is_day': 1,
  'condition': {'text': 'Partly cloudy',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png',
   'code': 1003},
  'wind_mph': 4.3,
  'wind_kph': 6.8,
  'wind_degree': 80,
  'wind_dir': 'E',
  'pressure_mb': 1011.0,
  'pressure_in': 29.85,
  'precip_mm': 0.13,
  'precip_in': 0.01,
  'humidity': 46,
  'cloud': 25,
  'feelslike_c': 33.2,
  'feelslike_f': 91.7,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 7.0,
  'gust_mph': 8.7,
  'gust_kph': 14.0},
 'forecast': {'forecastday': [{'date': '2024-01-15',
    'date_epoch': 1705276800,
    'day': {'maxtemp_c': 31.1,
     'maxtemp_f': 88.0,
     'mintemp_c': 14.9,
     'mintemp_f': 58.9,
   

In [4]:
response['forecast']['forecastday'][0]['hour'][14]

{'time_epoch': 1705345200,
 'time': '2024-01-15 14:00',
 'temp_c': 31.1,
 'temp_f': 88.0,
 'is_day': 1,
 'condition': {'text': 'Patchy rain possible',
  'icon': '//cdn.weatherapi.com/weather/64x64/day/176.png',
  'code': 1063},
 'wind_mph': 2.0,
 'wind_kph': 3.2,
 'wind_degree': 354,
 'wind_dir': 'N',
 'pressure_mb': 1007.0,
 'pressure_in': 29.73,
 'precip_mm': 0.06,
 'precip_in': 0.0,
 'humidity': 48,
 'cloud': 87,
 'feelslike_c': 32.9,
 'feelslike_f': 91.2,
 'windchill_c': 31.1,
 'windchill_f': 88.0,
 'heatindex_c': 32.9,
 'heatindex_f': 91.2,
 'dewpoint_c': 18.7,
 'dewpoint_f': 65.6,
 'will_it_rain': 1,
 'chance_of_rain': 100,
 'will_it_snow': 0,
 'chance_of_snow': 0,
 'vis_km': 10.0,
 'vis_miles': 6.0,
 'gust_mph': 2.3,
 'gust_kph': 3.7,
 'uv': 7.0}

In [5]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [6]:
response['forecast']['forecastday'][0]['hour'][0]['time']

'2024-01-15 00:00'

In [7]:
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[0]

'2024-01-15'

In [8]:
int(response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0])

1

In [9]:
response['forecast']['forecastday'][0]['hour'][14]['condition']

{'text': 'Patchy rain possible',
 'icon': '//cdn.weatherapi.com/weather/64x64/day/176.png',
 'code': 1063}

In [10]:
response['forecast']['forecastday'][0]['hour'][14]['condition']['text']

'Patchy rain possible'

In [11]:
response['forecast']['forecastday'][0]['hour'][14]['temp_c']

31.1

In [12]:
response['forecast']['forecastday'][0]['hour'][14]['will_it_rain']

1

In [13]:
response['forecast']['forecastday'][0]['hour'][14]['chance_of_rain']

100

# Dataframe

In [14]:
def get_forecast(response,i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = response['forecast']['forecastday'][0]['hour'][i]['temp_c']
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha, hora, condicion, tempe, rain, prob_rain

In [15]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour='green'):
    datos.append(get_forecast(response,i))

100%|██████████| 24/24 [00:00<00:00, 78581.81it/s]


In [16]:
datos

[('2024-01-15', 0, 'Patchy rain possible', 15.9, 1, 83),
 ('2024-01-15', 1, 'Patchy rain possible', 15.5, 1, 100),
 ('2024-01-15', 2, 'Patchy rain possible', 15.5, 1, 100),
 ('2024-01-15', 3, 'Patchy rain possible', 15.3, 1, 100),
 ('2024-01-15', 4, 'Mist', 15.2, 0, 0),
 ('2024-01-15', 5, 'Mist', 15.1, 0, 0),
 ('2024-01-15', 6, 'Partly cloudy', 14.9, 0, 0),
 ('2024-01-15', 7, 'Partly cloudy', 17.5, 0, 0),
 ('2024-01-15', 8, 'Cloudy', 21.8, 0, 0),
 ('2024-01-15', 9, 'Patchy rain possible', 23.2, 1, 76),
 ('2024-01-15', 10, 'Patchy rain possible', 25.4, 1, 100),
 ('2024-01-15', 11, 'Patchy rain possible', 28.0, 1, 84),
 ('2024-01-15', 12, 'Patchy rain possible', 29.8, 1, 87),
 ('2024-01-15', 13, 'Patchy rain possible', 30.9, 1, 87),
 ('2024-01-15', 14, 'Patchy rain possible', 31.1, 1, 100),
 ('2024-01-15', 15, 'Partly cloudy', 31.0, 1, 100),
 ('2024-01-15', 16, 'Patchy rain possible', 29.3, 1, 100),
 ('2024-01-15', 17, 'Patchy rain possible', 27.3, 0, 69),
 ('2024-01-15', 18, 'Patchy rai

In [17]:
col = ['fecha', 'hora', 'condicion', 'temperatura', 'lluvia', 'prob_lluvia']

df = pd.DataFrame(datos,columns=col)
df

,fecha,hora,condicion,temperatura,lluvia,prob_lluvia
0,2024-01-15,0,Patchy rain possible,15.9,1,83
1,2024-01-15,1,Patchy rain possible,15.5,1,100
2,2024-01-15,2,Patchy rain possible,15.5,1,100
3,2024-01-15,3,Patchy rain possible,15.3,1,100
4,2024-01-15,4,Mist,15.2,0,0
5,2024-01-15,5,Mist,15.1,0,0
6,2024-01-15,6,Partly cloudy,14.9,0,0
7,2024-01-15,7,Partly cloudy,17.5,0,0
8,2024-01-15,8,Cloudy,21.8,0,0
9,2024-01-15,9,Patchy rain possible,23.2,1,76


In [18]:
df_rain = df[(df['lluvia'] == 1) & (df['hora']> 6) & (df['hora'] < 22)]
df_rain = df_rain[['hora','condicion']]
df_rain.set_index('hora',inplace=True)

In [19]:
df_rain

,condicion
hora,
9,Patchy rain possible
10,Patchy rain possible
11,Patchy rain possible
12,Patchy rain possible
13,Patchy rain possible
14,Patchy rain possible
15,Partly cloudy
16,Patchy rain possible
18,Patchy rain possible


In [20]:
template = '\nHola!\n\n\n El pronostico del tiempo hoy ' + df['fecha'][0] + ' en ' + query + ' es: \n\n\n' + str(df_rain)
template

'\nHola!\n\n\n El pronostico del tiempo hoy 2024-01-15 en Cali es: \n\n\n                 condicion\nhora                      \n9     Patchy rain possible\n10    Patchy rain possible\n11    Patchy rain possible\n12    Patchy rain possible\n13    Patchy rain possible\n14    Patchy rain possible\n15           Partly cloudy\n16    Patchy rain possible\n18    Patchy rain possible\n19    Patchy rain possible\n20    Patchy rain possible\n21    Patchy rain possible'

In [21]:
PHONE_NUMBER

'+16067737564'

# Mensaje SMS desde TWILIO

In [24]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid,auth_token)
message = client.messages \
    .create(body=template,
           from_=PHONE_NUMBER,
           to='+5555555')

print('Mensaje Enviado '+ message.sid)

Mensaje Enviado 
